In [1]:
import pandas as pd
import pickle

In [2]:
# with open('extracted_features_2015.pkl', 'rb') as f:
#     ViT_2015_features = pickle.load(f)

# with open('extracted_features_2019.pkl', 'rb') as f:
#     ViT_2019_features = pickle.load(f)

In [3]:
# # Function to convert feature dictionary to a DataFrame
# def features_dict_to_df(features_dict, year):
#     # Initialize an empty DataFrame
#     df_features = pd.DataFrame()
    
#     # Loop through each key-value pair in the dictionary
#     for district_id, features in features_dict.items():
#         # Convert the feature array to a DataFrame
#         df_district = pd.DataFrame(features)
        
#         # Add a 'District' column with the current district ID
#         df_district['District'] = district_id
        
#         # Add a 'Year' column with the given year
#         df_district['Year'] = year
        
#         # Append the district DataFrame to the main DataFrame
#         df_features = pd.concat([df_features, df_district], ignore_index=True)
    
#     # Return the final DataFrame
#     return df_features

# # Convert the image feature dictionaries into DataFrames
# df_2015_features = features_dict_to_df(ViT_2015_features, 2015)
# df_2019_features = features_dict_to_df(ViT_2019_features, 2019)

In [4]:
# df_2015_features.head()

In [5]:
# df_2015_features = df_2015_features.drop_duplicates(subset='District')
# df_2019_features = df_2019_features.drop_duplicates(subset='District')

In [6]:
# df_features_combined = pd.concat([df_2015_features, df_2019_features], ignore_index=True)

In [7]:
# unique_districts_2015 = df_2015_features['District'].unique()

# # Filter the 2019 features to include only those districts present in 2015
# df_2019_features_filtered = df_2019_features[df_2019_features['District'].isin(unique_districts_2015)]


In [8]:
# df_2019_features_filtered.to_csv('vit_2019_features.csv', index=False)
# df_2015_features.to_csv('vit_2015_features.csv', index=False)

In [9]:
vit_2015_features = pd.read_csv('./vit_2015_features.csv')
vit_2019_features = pd.read_csv('./vit_2019_features.csv')

### Filtering and mergin datasets

In [10]:
tab_2015_19 = pd.read_csv('./final_2015_19.csv')

In [11]:
print(tab_2015_19)

       Country  Cluster Number  District     Case ID  Year  ELECTHH  COOKFUEL  \
0          IA6          150001       366    15000105  2015        1         8   
1          IA6          150001       366    15000113  2015        1         8   
2          IA6          150001       366    15000116  2015        0         8   
3          IA6          150001       366    15000120  2015        1         8   
4          IA6          150001       366    15000122  2015        1         8   
...        ...             ...       ...         ...   ...      ...       ...   
202741     IA7           50515       505  2750501549  2019        1         2   
202742     IA7           50515       505  2750501554  2019        1         2   
202743     IA7           50515       505  2750501544  2019        1         2   
202744     IA7           50515       505  2750501535  2019        1         2   
202745     IA7           50515       505  2750501592  2019        1         2   

        TOILETTYPE  DRINKWA

## Dimensionality Reduction

### PCA

In [12]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

In [23]:
from sklearn.decomposition import PCA

# Assume you have already loaded and scaled your ViT features into DataFrames named df_2015_features and df_2019_features

# Perform PCA
pca = PCA(n_components=88)  # or another number of components you have chosen based on the variance ratio plot
pca_2015_features = pca.fit_transform(vit_2015_features.drop(['District'], axis=1))
pca_2019_features = pca.fit_transform(vit_2019_features.drop(['District'], axis=1))


In [24]:
# Convert the PCA arrays into DataFrames
df_pca_2015_features = pd.DataFrame(pca_2015_features, columns=[f'PCA_{i+1}' for i in range(pca_2015_features.shape[1])])
df_pca_2019_features = pd.DataFrame(pca_2019_features, columns=[f'PCA_{i+1}' for i in range(pca_2019_features.shape[1])])

# Add the District column back to these DataFrames
df_pca_2015_features['District'] = vit_2015_features['District'].values
df_pca_2019_features['District'] = vit_2019_features['District'].values


In [25]:
# Merge the PCA features with the tabular data
tab_2015_19['Year'] = tab_2015_19['Year'].astype(int)  # Ensure that the Year column is an integer for proper merging

# Merge 2015 PCA features
merged_2015 = tab_2015_19[tab_2015_19['Year'] == 2015].merge(df_pca_2015_features, on='District', how='left')

# Merge 2019 PCA features
merged_2019 = tab_2015_19[tab_2015_19['Year'] == 2019].merge(df_pca_2019_features, on='District', how='left')

# Combine the merged data for both years
final_merged = pd.concat([merged_2015, merged_2019], ignore_index=True)


In [26]:
final_merged

,Country,Cluster Number,District,Case ID,Year,ELECTHH,COOKFUEL,TOILETTYPE,DRINKWATER,WEALTHQHH,...,PCA_79,PCA_80,PCA_81,PCA_82,PCA_83,PCA_84,PCA_85,PCA_86,PCA_87,PCA_88
0,IA6,150001,366,15000105,2015,1,8,31,21,1,...,-0.340670,-1.157812,-0.070379,5.553856e-15,-4.989976e-17,-1.402544e-17,1.253180e-17,1.286778e-17,8.018718e-33,5.785255e-17
1,IA6,150001,366,15000113,2015,1,8,31,32,1,...,-0.340670,-1.157812,-0.070379,5.553856e-15,-4.989976e-17,-1.402544e-17,1.253180e-17,1.286778e-17,8.018718e-33,5.785255e-17
2,IA6,150001,366,15000116,2015,0,8,31,32,1,...,-0.340670,-1.157812,-0.070379,5.553856e-15,-4.989976e-17,-1.402544e-17,1.253180e-17,1.286778e-17,8.018718e-33,5.785255e-17
3,IA6,150001,366,15000120,2015,1,8,31,13,2,...,-0.340670,-1.157812,-0.070379,5.553856e-15,-4.989976e-17,-1.402544e-17,1.253180e-17,1.286778e-17,8.018718e-33,5.785255e-17
4,IA6,150001,366,15000122,2015,1,8,31,32,1,...,-0.340670,-1.157812,-0.070379,5.553856e-15,-4.989976e-17,-1.402544e-17,1.253180e-17,1.286778e-17,8.018718e-33,5.785255e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202741,IA7,50515,505,2750501549,2019,1,2,12,11,5,...,0.021041,-0.018911,-0.023892,5.022526e-15,-5.514029e-17,-3.604740e-15,-2.550691e-15,-8.133099e-16,-1.055246e-15,5.448141e-15
202742,IA7,50515,505,2750501554,2019,1,2,12,11,4,...,0.021041,-0.018911,-0.023892,5.022526e-15,-5.514029e-17,-3.604740e-15,-2.550691e-15,-8.133099e-16,-1.055246e-15,5.448141e-15
202743,IA7,50515,505,2750501544,2019,1,2,12,32,4,...,0.021041,-0.018911,-0.023892,5.022526e-15,-5.514029e-17,-3.604740e-15,-2.550691e-15,-8.133099e-16,-1.055246e-15,5.448141e-15
202744,IA7,50515,505,2750501535,2019,1,2,12,11,4,...,0.021041,-0.018911,-0.023892,5.022526e-15,-5.514029e-17,-3.604740e-15,-2.550691e-15,-8.133099e-16,-1.055246e-15,5.448141e-15


In [27]:
final_merged.to_csv('final_2015_19_with_ViT.csv', index=False)

In [20]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# # Assuming 'final_merged' is your DataFrame after merging the PCA features

# # Separate the features and target variable
# X = final_merged.drop('WEALTHSHH', axis=1)  # Replace 'target_column' with the actual name of your target column
# y = final_merged['WEALTHSHH']  # Replace 'target_column' with the actual name of your target column

# # List of numerical and categorical features
# numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
# categorical_features = X.select_dtypes(include=['object']).columns

# # Creating the preprocessing pipelines for both numeric and categorical data
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),  # Imputing missing values with the mean
#     ('scaler', StandardScaler())  # Scaling features
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Handling missing categorical data
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One hot encoding for categorical variables
# ])

# # Combining transformers into a preprocessor with ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numerical_features),
#         ('cat', categorical_transformer, categorical_features)
#     ])

# # Splitting data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Applying the preprocessing pipeline to the training data
# X_train_preprocessed = preprocessor.fit_transform(X_train)

# # Applying the preprocessing pipeline to the test data
# X_test_preprocessed = preprocessor.transform(X_test)

# Now, X_train_preprocessed and X_test_preprocessed are ready for modeling
